## Install KenLM

```bash
sudo apt -y install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
ls kenlm/build/bin
```

In [1]:
from datasets import load_dataset

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
ds = load_dataset("bigcode/the-stack-pjjs-medium", use_auth_token=True, split="train")

Using custom data configuration bigcode--the-stack-pjjs-medium-d5c5ce7928f966c0
Found cached dataset json (/home/leandro/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-pjjs-medium-d5c5ce7928f966c0/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


In [8]:
from transformers import AutoTokenizer

In [70]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/digit-bytelevel-bpe-jss-v1.1-49152", use_auth_token=True, slow=True)

In [7]:
ds

Dataset({
    features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'],
    num_rows: 600000
})

In [80]:
!rm samples.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [81]:
def save_tokens(examples):
    tokenized_content = tokenizer(examples["content"])
    with open('samples.txt', 'a') as f:
        for input_ids in tokenized_content.input_ids:
            f.write(" ".join(tokenizer.convert_ids_to_tokens(input_ids)))
            f.write("\n")

In [82]:
_ = ds.map(save_tokens, batched=True, batch_size=1000)

  0%|          | 0/600 [00:00<?, ?ba/s]

```bash
kenlm/build/bin/lmplz -o 5 < "samples.txt" > "gram.arpa"
kenlm/build/bin/build_binary gram.arpa gram.binary
```

In [84]:
import kenlm
model = kenlm.LanguageModel('./gram.binary')

In [85]:
model.score("this is a test")

-21.19799041748047